### Importação de bibliotecas

In [1]:
import pandas as pd
import pathlib


### Importação do arquivo fonte de dados

In [2]:
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
email = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding = 'latin1', sep=';')


vendas = vendas.merge(lojas, on='ID Loja') #inserir o nome da loja na tabela vendas, conforme o ID da Loja.


vendas.info()
lojas.info()
email.info()

<class 'pandas.DataFrame'>
RangeIndex: 100999 entries, 0 to 100998
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Código Venda    100999 non-null  int64         
 1   Data            100999 non-null  datetime64[us]
 2   ID Loja         100999 non-null  int64         
 3   Produto         100999 non-null  str           
 4   Quantidade      100999 non-null  int64         
 5   Valor Unitário  100999 non-null  int64         
 6   Valor Final     100999 non-null  int64         
 7   Loja            100999 non-null  str           
dtypes: datetime64[us](1), int64(5), str(2)
memory usage: 6.2 MB
<class 'pandas.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   ID Loja  25 non-null     int64
 1   Loja     25 non-null     str  
dtypes: int64(1), str(1)
memory usage: 532.0 bytes
<class 'pandas.DataF

### Criação das tabelas de cada loja

In [3]:
dic_lojas = {}

for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja'] == loja, :] # os ':" representam que quero todas as colunas que as linhas tenham a loja

dict_keys = dic_lojas.keys()

print(dict_keys)

dict_keys(['Iguatemi Esplanada', 'Shopping Midway Mall', 'Norte Shopping', 'Shopping Iguatemi Fortaleza', 'Shopping União de Osasco', 'Shopping Center Interlagos', 'Rio Mar Recife', 'Salvador Shopping', 'Rio Mar Shopping Fortaleza', 'Shopping Center Leste Aricanduva', 'Ribeirão Shopping', 'Shopping Morumbi', 'Parque Dom Pedro Shopping', 'Bourbon Shopping SP', 'Palladium Shopping Curitiba', 'Passei das Águas Shopping', 'Center Shopping Uberlândia', 'Shopping Recife', 'Shopping Vila Velha', 'Shopping SP Market', 'Shopping Eldorado', 'Shopping Ibirapuera', 'Novo Shopping Ribeirão Preto', 'Iguatemi Campinas', 'Shopping Barra'])


### Salvar os arquivos individuais

In [4]:
# identifica se a pasta existe, caso não, cria a pasta

caminho = pathlib.Path(r'Backup Arquivos Lojas')

data_salvar = vendas['Data'].max()

arquivo_pasta = caminho.iterdir()
nome_arquivos = [arquivo.name for arquivo in arquivo_pasta]

for loja in dic_lojas:
    if loja not in nome_arquivos:
        pasta = caminho / loja
        pasta.mkdir()

# salva o arquivo dentro da pasta com o nome da loja

arquivo = '{}_{}_{}.xlsx'.format(data_salvar.month, data_salvar.day, loja) # gerou o nome do arquivo
local = caminho / loja / arquivo # determina o local para salvar o arquivo 

dic_lojas[loja].to_excel(local) # criou os arquivos de cada loja, dentro das pastas de cada loja


### Estabelecer as metas

In [5]:
meta_faturamento_ano = 1650000
meta_faturamento_dia = 1000
meta_produtos_ano =120  # (quantos produtos diferentes foram vendidos naquele período)
meta_produtos_dia =4
meta_ticket_ano = 500
meta_ticket_dia = 500

### Determinar os indicadores para cada loja

In [ ]:
for loja in dic_lojas:
    dados_loja = dic_lojas[loja]
    venda_loja_ano = dados_loja['Valor Final'].sum()
    venda_loja_dia = dados_loja.loc[dados_loja['Data']==data_salvar]['Valor Final'].sum()
    produto_loja_ano = len(dados_loja['Produto'].unique())
    produto_loja_dia = len(dados_loja.loc[dados_loja['Data']==data_salvar]['Produto'].unique())
    valor_por_venda = dados_loja.groupby('Código Venda').sum(numeric_only=True)
    valor_por_venda_dia = dados_loja.loc[dados_loja['Data']==data_salvar].groupby('Código Venda').sum(numeric_only=True)
    ticket_loja_ano = valor_por_venda['Valor Final'].mean()
    ticket_loja_dia = valor_por_venda_dia['Valor Final'].mean()

    print(venda_loja_ano)
    print(venda_loja_dia)
    print(produto_loja_ano)
    print(produto_loja_dia)
    print(ticket_loja_ano)
    print(ticket_loja_dia)


1699681
0
120
0
779.0306314797361
nan
1590441
170
120
1
779.0306314797361
170.0
1711968
1259
120
2
779.0306314797361
1259.0
1674824
0
120
0
779.0306314797361
nan
1663770
1207
120
3
779.0306314797361
1207.0
1640486
1582
120
2
779.0306314797361
1582.0
1722766
0
120
0
779.0306314797361
nan
1646739
3950
120
2
779.0306314797361
3950.0
1698430
368
120
1
779.0306314797361
368.0
1682870
485
120
2
779.0306314797361
485.0
1707122
650
120
3
779.0306314797361
325.0
1586444
0
120
0
779.0306314797361
nan
1631630
0
120
0
779.0306314797361
nan
1726110
676
120
1
779.0306314797361
676.0
1721120
381
120
2
779.0306314797361
190.5
1649014
0
120
0
779.0306314797361
nan
1668921
2651
120
4
779.0306314797361
1325.5
1624879
1366
120
3
779.0306314797361
683.0
1731167
937
120
3
779.0306314797361
312.3333333333333
1721763
0
120
0
779.0306314797361
nan
1648111
2391
120
2
779.0306314797361
2391.0
1635060
118
120
1
779.0306314797361
118.0
1678225
3400
120
6
779.0306314797361
1700.0
1762419
0
120
0
779.0306314797361
n